In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pointbiserialr
from scipy.stats import skew, kurtosis
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, recall_score, classification_report, confusion_matrix
from imblearn.metrics import specificity_score



Dane

In [ ]:
data = pd.read_csv('data/winequality-white.csv', delimiter=';')

In [ ]:
types_temp = [i % 2 for i in range(len(data))]
data['type'] = types_temp

In [ ]:
data.head()

OPIS I WSTEPNA ANALIZA DANYCH

Typy danych w zbiorze

In [ ]:
data.dtypes

Rozmiar danych

In [ ]:
data.shape

Statystyki opisowe dla zmiennych liczbowych

In [ ]:
data_float_column = data.select_dtypes(include=[float]).columns
data[data_float_column].describe()

Analiza Wpływu Zmiennych na Zmienną Prognozowaną (Y)


In [ ]:
#heat mapa korelacji  ale nie wiem czy to dobra metoda
plt.figure(figsize=(10,8))
sns.heatmap(data.corr(), annot=True, fmt='.2f')
plt.show()

In [ ]:
# Przypuśćmy, że 'df' to twój DataFrame, a 'quality' to twoja zmienna binarna.
# Przeiteruj przez wszystkie kolumny (oprócz 'quality') i oblicz korelację punktowo-biserialną.
for column in data.columns.drop('quality'):
    pbc = pointbiserialr(data[column], data['quality'])
    print(f"Korelacja punktowo-biserialna między {column} a quality: {pbc.correlation:.3f}, p-wartość: {pbc.pvalue:.4f}")


Ocena zbalansowania zbioru danych

In [ ]:
#Histogram
plt.figure(figsize=(8,4))
sns.histplot(data['quality'], kde=True)
plt.title('Histogram zmiennej quality')
plt.show()

In [ ]:
#Boxplot
plt.figure(figsize=(6, 7))
sns.boxplot(y=data['quality'])
plt.title('Boxplot zmiennej quality')
plt.show()

In [ ]:
#Skośnosc i kurtoza
print(f"Skośność: ", skew(data['quality']))
print(f'Kurtoza: ', kurtosis(data['quality']))

PRZYGOTOWANIE DANYCH

Wartosci brakujace

In [ ]:
data.isna().sum()

Sprawdzenie wartosci odstajacych

In [ ]:
# fig, axs = plt.subplots(ncols=3, nrows = 3, figsize=(15,15))
# 
# sns.boxplot(ax=axs[0][0], y=data['fixed acidity'])
# axs[0][0].set_title('fixed acidity')
# 
# sns.boxplot(ax=axs[1], y=data['volatile acidity'])
# axs[1].set_title('volatile acidity')
# 
# sns.boxplot(ax=axs[2], y=data['citric acid'])
# axs[2].set_title('citric acid')
# 
# sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})
# plt.tight_layout()
# plt.show()

PODZIAL DANYCH NA ZBIOR UCZACY I TESTOWY


In [ ]:
X = data.drop('quality', axis=1)
y = data['quality']

In [ ]:
#podzial na zbior testowy i treningowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#kopia danych
X_train_copy = X_train.copy()
X_test_copy = X_test.copy()

Statystyki opisowe dla zbiorow

In [ ]:
X_train.describe()

In [ ]:
X_test.describe()


ALGORYTM SVM

Standaryzacja zmiennych

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
def svm_algorithm(kernel):
    model = svm.SVC(kernel=kernel)
    model.fit(X_train, y_train)
    
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    accuracy_train = accuracy_score(y_train, y_pred_train)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    recall_train = recall_score(y_train, y_pred_train,  average=None)
    recall_test = recall_score(y_test, y_pred_test, average=None)
    specificity_train = specificity_score(y_train, y_pred_train)
    specificity_test = specificity_score(y_test, y_pred_test)

    measures = ['', 'Dokładność', 'Czułość', 'Specyficzność']    
    print(f'Jądro: {kernel}')
    print(f'{measures[0]: <10} {measures[1]: <15} {measures[2]: <20} {measures[3]: <25}')
    print(f'Treningowy {round(accuracy_train, 3): <15} {round(recall_train, 3): <20} {round(specificity_train, 3): <25}')
    print(f'Testowy    {round(accuracy_test, 3): <15} {round(recall_test, 3): <20} {round(specificity_test, 3): <25}')
    print()

    
    

In [ ]:
svm_algorithm('rbf')